In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM

c:\Users\Mintz\miniconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("HebArabNlpProject/HebNLI")
models = {
    "AlephBERT": "onlplab/alephbert-base", # https://huggingface.co/onlplab/alephbert-base
    "mBERT": "bert-base-multilingual-cased", #https://huggingface.co/google-bert/bert-base-multilingual-cased
    "DictaBERT": "dicta-il/dictabert", # https://huggingface.co/dicta-il/dictabert  
}

In [3]:
# check all working:
for k, v in models.items():
    tokenizer = AutoTokenizer.from_pretrained(v)
    model = AutoModelForMaskedLM.from_pretrained(v)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
